In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout,Conv3D,Conv1D,Conv2D,Flatten,MaxPool2D
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/prop-3.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0      prop-3      225    7    5    0   21   23     9   0  21  ...  0.000000   
1      prop-3      225    4    3    0    5   16     4   0   5  ...  0.333333   
2      prop-3      225    3    3    0   21   28     3   0  21  ...  0.000000   
3      prop-3      225    5    5    0   23   28     4   0  23  ...  0.000000   
4      prop-3      225    5    4    0   14   21     4   1  13  ...  0.000000   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
10269  prop-3      265    2    3    0   10    8     1   1   9  ...  0.000000   
10270  prop-3      265    7    2    0   29   19    21  22   7  ...  0.000000   
10271  prop-3      265    2    3    0   10    8     1   1   9  ...  0.000000   
10272  prop-3      265    3    3    0   29   42     3   1  28  ...  0.000000   
10273  prop-3      265    2    1    0   78   80     1   0  78  ...  1.000000   

       moa       mfa       cam  ic  cbm         amc  max_cc  avg_cc  bug  
0        0  0.827586  0.466667   3    4   11.000000       1  0.7143    0  
1        0  0.909091  0.666667   1    2   14.750000       2  0.7500    0  
2        0  0.750000  0.444444   1    1   43.666667       7  3.3333    0  
3        0  0.888889  0.666667   2    3   23.800000       1  0.6000    0  
4        0  0.888889  0.583333   3    5   17.600000       1  0.6000    0  
...    ...       ...       ...  ..  ...         ...     ...     ...  ...  
10269    0  0.857143  0.625000   1    1   11.500000       2  1.0000    0  
10270    0  0.400000  0.500000   1    1   28.857143      12  2.4286    0  
10271    0  0.857143  0.666667   1    1   10.500000       1  0.5000    0  
10272    0  0.750000  0.416667   1    1   60.000000      12  4.6667    0  
10273    0  0.000000  1.000000   0    0  160.000000       0  0.0000    0  

[10274 rows x 23 columns]

In [4]:
ant_master = ant_master.loc[ant_master['version'] == 225]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0     prop-3      225    7    5    0   21   23     9   0  21  ...  0.000000   
1     prop-3      225    4    3    0    5   16     4   0   5  ...  0.333333   
2     prop-3      225    3    3    0   21   28     3   0  21  ...  0.000000   
3     prop-3      225    5    5    0   23   28     4   0  23  ...  0.000000   
4     prop-3      225    5    4    0   14   21     4   1  13  ...  0.000000   
...      ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
1805  prop-3      225    1    1    0    1    5     0   0   1  ...  0.000000   
1806  prop-3      225    3    1    0    9   30     3   0   9  ...  0.000000   
1807  prop-3      225    1    1    0    0    5     0   0   0  ...  0.000000   
1808  prop-3      225    1    1    0    0    5     0   0   0  ...  0.000000   
1809  prop-3      225    2    4    0   17   15     1   0  17  ...  0.000000   

      moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0       0  0.827586  0.466667   3    4  11.000000       1  0.7143    0  
1       0  0.909091  0.666667   1    2  14.750000       2  0.7500    0  
2       0  0.750000  0.444444   1    1  43.666667       7  3.3333    0  
3       0  0.888889  0.666667   2    3  23.800000       1  0.6000    0  
4       0  0.888889  0.583333   3    5  17.600000       1  0.6000    0  
...   ...       ...       ...  ..  ...        ...     ...     ...  ...  
1805    0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
1806    0  0.000000  0.466667   0    0  65.333333       8  4.0000    0  
1807    0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
1808    0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
1809    0  0.888889  0.666667   2    2  34.500000       1  0.5000    0  

[1810 rows x 23 columns]

In [5]:
train, test = train_test_split(ant_master, test_size=0.2)

In [6]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [7]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc  dam  moa  \
1508    5    1    0    2    6     6   2   0    5  0.750000   24  1.0    0   
73      2    4    0   15   14     1   0  15    2  2.000000   56  0.0    0   
759     2    4    0   32   36     1   0  32    2  2.000000  233  1.0    0   
542     7    6    0   13   19    15   0  13    2  0.944444   74  0.0    0   
458     5    5    0    4   15    10   0   4    1  2.000000   44  0.0    0   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...  ...  ...   
688     4    5    0    9   13     4   0   9    1  1.000000   43  0.0    0   
888     6    5    0   17   23    13   0  17    3  1.000000   95  0.0    0   
795     7    5    0   10   17    21   1   9    2  1.055556   61  0.0    0   
1502    5    4    0    9   15    10   0   9    1  2.000000   46  0.0    0   
79      2    3    0    6    6     1   0   6    2  2.000000   15  0.0    0   

           mfa       cam  ic  cbm         amc  max_cc  avg_cc  
1508  0.000000  0.700000   0    0    3.400000       1  0.8000  
73    0.888889  0.666667   2    2   27.000000       1  0.5000  
759   0.916667  0.625000   1    1  114.000000       1  0.5000  
542   0.848485  0.555556   3    7    9.142857       1  0.7143  
458   0.957746  1.000000   1    1    7.800000       1  0.6000  
...        ...       ...  ..  ...         ...     ...     ...  
688   0.923077  0.555556   2    2    9.000000       1  0.5000  
888   0.857143  0.360000   2    3   14.333333       2  0.8333  
795   0.895833  0.444444   4    8    7.285714       2  0.8571  
1502  0.764706  0.360000   2    4    8.200000       1  0.8000  
79    0.857143  0.600000   1    1    6.500000       1  0.5000  

[1448 rows x 20 columns]

In [8]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm  lcom3  loc  dam  moa  \
570     7    5    0   16   31    19   0  16    2    1.0   96  0.0    0   
1381    2    4    0   30   33     1   0  30    2    1.0  141  0.0    0   
233     2    2    0   12   14     1   0  12    2    2.0   48  0.0    0   
1416    1    3   10   11    2     0  10   1    1    2.0    4  0.0    0   
19      2    4    0   23   25     1   0  23    2    2.0  160  0.0    0   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...    ...  ...  ...  ...   
132     9    6    0    7   17    36   0   7    1    2.0   46  0.0    0   
463     2    3    0   12   11     1   0  12    2    2.0   36  0.0    0   
537     2    2    0    7   17     1   0   7    2    2.0   60  1.0    0   
830     3    4    0    4    5     3   0   4    1    2.0   12  0.0    0   
1181    5    1    0    2   11     0   1   1    5    0.0   35  1.0    0   

           mfa       cam  ic  cbm        amc  max_cc  avg_cc  
570   0.838710  0.400000   3    8  12.285714       3  1.0000  
1381  0.916667  0.625000   1    1  69.000000       1  0.5000  
233   0.800000  0.666667   1    1  23.000000       1  0.5000  
1416  1.000000  1.000000   0    0   3.000000       0  0.0000  
19    0.916667  0.625000   2    2  79.000000       1  0.5000  
...        ...       ...  ..  ...        ...     ...     ...  
132   0.940171  0.625000   2    3   4.111111       1  0.7778  
463   0.857143  0.625000   1    1  17.000000       2  1.0000  
537   0.800000  0.666667   1    1  28.000000       1  0.5000  
830   0.857143  0.666667   2    2   3.000000       1  0.6667  
1181  0.000000  0.533333   0    0   5.800000       1  0.8000  

[362 rows x 20 columns]

In [9]:
y_train = pd.DataFrame(train['bug'])
y_test = pd.DataFrame(test['bug'])

In [10]:
y_train

bug
1508    0
73      0
759     0
542     0
458     0
...   ...
688     0
888     0
795     2
1502    0
79      0

[1448 rows x 1 columns]

In [11]:
y_test

bug
570     0
1381    0
233     0
1416    0
19      0
...   ...
132     0
463     0
537     0
830     0
1181    0

[362 rows x 1 columns]

In [12]:
#Getting the rows and columns size in our data
img_rows, img_cols = 1,20
# Preparing the data for the model.
X_train_matrix = x_train.values
X_test_matrix = x_test.values
Y_train_matrix = y_train.values
Y_test_matrix = y_test.values
# Preparing the data for the model.
Ytrainseries = y_train['bug']
Ytestseries = y_test['bug']

In [13]:
# Preparing the data for the model.
X_train1 = X_train_matrix.reshape(X_train_matrix.shape[0], img_rows, img_cols, 1)
X_test1 = X_test_matrix.reshape(X_test_matrix.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [14]:
def define_model():
    # Designing and initializing the model.
    #Building the model
    model = Sequential()
    #add model layers
    model.add(Conv1D(2, kernel_size=2, activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(Dense(1, activation='relu'))
    #compile model using mse as the loss function
    model.compile(loss='mse', optimizer = keras.optimizers.Adam(learning_rate=0.0005), metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()])
    return model

In [15]:
y = y_test['bug'].tolist()
y

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [16]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(30):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/cnn_propv225/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(X_train1, Y_train_matrix, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(X_test1, Y_test_matrix)
    total_scores.append(score) 
    predictions = model.predict(X_test1)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
15/15 [==============================] - 0s 2ms/step - loss: 0.3142 - mse: 0.3142 - mae: 0.1333 - root_mean_squared_error: 0.5606 - mean_squared_logarithmic_error: 0.0765
Epoch 2/100
15/15 [==============================] - 0s 1ms/step - loss: 0.3142 - mse: 0.3142 - mae: 0.1333 - root_mean_squared_error: 0.5606 - mean_squared_logarithmic_error: 0.0765
Epoch 3/100
15/15 [==============================] - 0s 1ms/step - loss: 0.3142 - mse: 0.3142 - mae: 0.1333 - root_mean_squared_error: 0.5606 - mean_squared_logarithmic_error: 0.0765
Epoch 4/100
15/15 [==============================] - 0s 1ms/step - loss: 0.3142 - mse: 0.3142 - mae: 0.1333 - root_mean_squared_error: 0.5606 - mean_squared_logarithmic_error: 0.0765
Epoch 5/100
15/15 [==============================] - 0s 1ms/step - loss: 0.3142 - mse: 0.3142 - mae: 0.1333 - root_mean_squared_error: 0.5606 - mean_squared_logarithmic_error: 0.0765
Epoch 6/100
15/15 [==============================] - 0s 1ms/step - loss: 0.

15/15 [==============================] - 0s 940us/step - loss: 0.3142 - mse: 0.3142 - mae: 0.1333 - root_mean_squared_error: 0.5606 - mean_squared_logarithmic_error: 0.0765
Epoch 88/100
15/15 [==============================] - 0s 977us/step - loss: 0.3142 - mse: 0.3142 - mae: 0.1333 - root_mean_squared_error: 0.5606 - mean_squared_logarithmic_error: 0.0765
Epoch 89/100
15/15 [==============================] - 0s 1ms/step - loss: 0.3142 - mse: 0.3142 - mae: 0.1333 - root_mean_squared_error: 0.5606 - mean_squared_logarithmic_error: 0.0765
Epoch 90/100
15/15 [==============================] - 0s 1ms/step - loss: 0.3142 - mse: 0.3142 - mae: 0.1333 - root_mean_squared_error: 0.5606 - mean_squared_logarithmic_error: 0.0765
Epoch 91/100
15/15 [==============================] - 0s 1ms/step - loss: 0.3142 - mse: 0.3142 - mae: 0.1333 - root_mean_squared_error: 0.5606 - mean_squared_logarithmic_error: 0.0765
Epoch 92/100
15/15 [==============================] - 0s 972us/step - loss: 0.3142 - mse:

15/15 [==============================] - 0s 1ms/step - loss: 0.7445 - mse: 0.7445 - mae: 0.1623 - root_mean_squared_error: 0.8629 - mean_squared_logarithmic_error: 0.0913
Epoch 73/100
15/15 [==============================] - 0s 999us/step - loss: 0.7316 - mse: 0.7316 - mae: 0.1617 - root_mean_squared_error: 0.8553 - mean_squared_logarithmic_error: 0.0910
Epoch 74/100
15/15 [==============================] - 0s 878us/step - loss: 0.7190 - mse: 0.7190 - mae: 0.1610 - root_mean_squared_error: 0.8480 - mean_squared_logarithmic_error: 0.0906
Epoch 75/100
15/15 [==============================] - 0s 1ms/step - loss: 0.7055 - mse: 0.7055 - mae: 0.1601 - root_mean_squared_error: 0.8399 - mean_squared_logarithmic_error: 0.0901
Epoch 76/100
15/15 [==============================] - 0s 1ms/step - loss: 0.6912 - mse: 0.6912 - mae: 0.1594 - root_mean_squared_error: 0.8314 - mean_squared_logarithmic_error: 0.0898
Epoch 77/100
15/15 [==============================] - 0s 1ms/step - loss: 0.6825 - mse: 0

15/15 [==============================] - 0s 1ms/step - loss: 0.6992 - mse: 0.6992 - mae: 0.1819 - root_mean_squared_error: 0.8362 - mean_squared_logarithmic_error: 0.0987
Epoch 58/100
15/15 [==============================] - 0s 1ms/step - loss: 0.6806 - mse: 0.6806 - mae: 0.1800 - root_mean_squared_error: 0.8250 - mean_squared_logarithmic_error: 0.0979
Epoch 59/100
15/15 [==============================] - 0s 1ms/step - loss: 0.6615 - mse: 0.6615 - mae: 0.1784 - root_mean_squared_error: 0.8134 - mean_squared_logarithmic_error: 0.0971
Epoch 60/100
15/15 [==============================] - 0s 921us/step - loss: 0.6468 - mse: 0.6468 - mae: 0.1768 - root_mean_squared_error: 0.8042 - mean_squared_logarithmic_error: 0.0963
Epoch 61/100
15/15 [==============================] - 0s 1ms/step - loss: 0.6280 - mse: 0.6280 - mae: 0.1753 - root_mean_squared_error: 0.7924 - mean_squared_logarithmic_error: 0.0957
Epoch 62/100
15/15 [==============================] - 0s 1ms/step - loss: 0.6125 - mse: 0.6

15/15 [==============================] - 0s 1ms/step - loss: 3.9636 - mse: 3.9636 - mae: 0.5771 - root_mean_squared_error: 1.9909 - mean_squared_logarithmic_error: 0.3133
Epoch 43/100
15/15 [==============================] - 0s 1ms/step - loss: 3.7160 - mse: 3.7160 - mae: 0.5595 - root_mean_squared_error: 1.9277 - mean_squared_logarithmic_error: 0.3042
Epoch 44/100
15/15 [==============================] - 0s 1ms/step - loss: 3.5530 - mse: 3.5530 - mae: 0.5431 - root_mean_squared_error: 1.8849 - mean_squared_logarithmic_error: 0.2956
Epoch 45/100
15/15 [==============================] - 0s 1ms/step - loss: 3.3415 - mse: 3.3415 - mae: 0.5273 - root_mean_squared_error: 1.8280 - mean_squared_logarithmic_error: 0.2876
Epoch 46/100
15/15 [==============================] - 0s 1ms/step - loss: 3.2016 - mse: 3.2016 - mae: 0.5143 - root_mean_squared_error: 1.7893 - mean_squared_logarithmic_error: 0.2806
Epoch 47/100
15/15 [==============================] - 0s 970us/step - loss: 3.0488 - mse: 3.0

...

15/15 [==============================] - 0s 1ms/step - loss: 0.3142 - mse: 0.3142 - mae: 0.1333 - root_mean_squared_error: 0.5606 - mean_squared_logarithmic_error: 0.0765
Epoch 44/100
15/15 [==============================] - 0s 1ms/step - loss: 0.3142 - mse: 0.3142 - mae: 0.1333 - root_mean_squared_error: 0.5606 - mean_squared_logarithmic_error: 0.0765
Epoch 45/100
15/15 [==============================] - 0s 1ms/step - loss: 0.3142 - mse: 0.3142 - mae: 0.1333 - root_mean_squared_error: 0.5606 - mean_squared_logarithmic_error: 0.0765
Epoch 46/100
15/15 [==============================] - 0s 1ms/step - loss: 0.3142 - mse: 0.3142 - mae: 0.1333 - root_mean_squared_error: 0.5606 - mean_squared_logarithmic_error: 0.0765
Epoch 47/100
15/15 [==============================] - 0s 1ms/step - loss: 0.3142 - mse: 0.3142 - mae: 0.1333 - root_mean_squared_error: 0.5606 - mean_squared_logarithmic_error: 0.0765
Epoch 48/100
15/15 [==============================] - 0s 1ms/step - loss: 0.3142 - mse: 0.314

15/15 [==============================] - 0s 1ms/step - loss: 0.5203 - mse: 0.5203 - mae: 0.1520 - root_mean_squared_error: 0.7213 - mean_squared_logarithmic_error: 0.0858
Epoch 30/100
15/15 [==============================] - 0s 1ms/step - loss: 0.5126 - mse: 0.5126 - mae: 0.1513 - root_mean_squared_error: 0.7160 - mean_squared_logarithmic_error: 0.0855
Epoch 31/100
15/15 [==============================] - 0s 1ms/step - loss: 0.5077 - mse: 0.5077 - mae: 0.1508 - root_mean_squared_error: 0.7126 - mean_squared_logarithmic_error: 0.0853
Epoch 32/100
15/15 [==============================] - 0s 1ms/step - loss: 0.5038 - mse: 0.5038 - mae: 0.1501 - root_mean_squared_error: 0.7098 - mean_squared_logarithmic_error: 0.0852
Epoch 33/100
15/15 [==============================] - 0s 1ms/step - loss: 0.4960 - mse: 0.4960 - mae: 0.1496 - root_mean_squared_error: 0.7042 - mean_squared_logarithmic_error: 0.0850
Epoch 34/100
15/15 [==============================] - 0s 1ms/step - loss: 0.4910 - mse: 0.491

15/15 [==============================] - 0s 1ms/step - loss: 0.3142 - mse: 0.3142 - mae: 0.1344 - root_mean_squared_error: 0.5605 - mean_squared_logarithmic_error: 0.0765
Epoch 16/100
15/15 [==============================] - 0s 1ms/step - loss: 0.3143 - mse: 0.3143 - mae: 0.1340 - root_mean_squared_error: 0.5606 - mean_squared_logarithmic_error: 0.0766
Epoch 17/100
15/15 [==============================] - 0s 1ms/step - loss: 0.3140 - mse: 0.3140 - mae: 0.1340 - root_mean_squared_error: 0.5604 - mean_squared_logarithmic_error: 0.0764
Epoch 18/100
15/15 [==============================] - 0s 1ms/step - loss: 0.3140 - mse: 0.3140 - mae: 0.1341 - root_mean_squared_error: 0.5604 - mean_squared_logarithmic_error: 0.0764
Epoch 19/100
15/15 [==============================] - 0s 1ms/step - loss: 0.3141 - mse: 0.3141 - mae: 0.1340 - root_mean_squared_error: 0.5605 - mean_squared_logarithmic_error: 0.0765
Epoch 20/100
15/15 [==============================] - 0s 1ms/step - loss: 0.3139 - mse: 0.313

15/15 [==============================] - 0s 5ms/step - loss: 307.3281 - mse: 307.3281 - mae: 9.0249 - root_mean_squared_error: 17.5308 - mean_squared_logarithmic_error: 3.9229
Epoch 2/100
15/15 [==============================] - 0s 1ms/step - loss: 208.8811 - mse: 208.8811 - mae: 7.1421 - root_mean_squared_error: 14.4527 - mean_squared_logarithmic_error: 3.2064
Epoch 3/100
15/15 [==============================] - 0s 1ms/step - loss: 139.0500 - mse: 139.0500 - mae: 5.5596 - root_mean_squared_error: 11.7919 - mean_squared_logarithmic_error: 2.5692
Epoch 4/100
15/15 [==============================] - 0s 1ms/step - loss: 95.8193 - mse: 95.8193 - mae: 4.2795 - root_mean_squared_error: 9.7887 - mean_squared_logarithmic_error: 2.0126
Epoch 5/100
15/15 [==============================] - 0s 957us/step - loss: 68.2886 - mse: 68.2886 - mae: 3.2540 - root_mean_squared_error: 8.2637 - mean_squared_logarithmic_error: 1.5502
Epoch 6/100
15/15 [==============================] - 0s 1ms/step - loss: 48.

15/15 [==============================] - 0s 1ms/step - loss: 0.4762 - mse: 0.4762 - mae: 0.1541 - root_mean_squared_error: 0.6901 - mean_squared_logarithmic_error: 0.0877
Epoch 90/100
15/15 [==============================] - 0s 1ms/step - loss: 0.4739 - mse: 0.4739 - mae: 0.1538 - root_mean_squared_error: 0.6884 - mean_squared_logarithmic_error: 0.0876
Epoch 91/100
15/15 [==============================] - 0s 991us/step - loss: 0.4722 - mse: 0.4722 - mae: 0.1536 - root_mean_squared_error: 0.6872 - mean_squared_logarithmic_error: 0.0874
Epoch 92/100
15/15 [==============================] - 0s 1ms/step - loss: 0.4697 - mse: 0.4697 - mae: 0.1534 - root_mean_squared_error: 0.6854 - mean_squared_logarithmic_error: 0.0873
Epoch 93/100
15/15 [==============================] - 0s 1ms/step - loss: 0.4679 - mse: 0.4679 - mae: 0.1532 - root_mean_squared_error: 0.6840 - mean_squared_logarithmic_error: 0.0872
Epoch 94/100
15/15 [==============================] - 0s 922us/step - loss: 0.4662 - mse: 0

In [17]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/cnn_propv225/'

In [18]:
total_training_time

[2.565032958984375,
 2.3436076641082764,
 2.3601677417755127,
 2.3715660572052,
 2.3884451389312744,
 2.1766161918640137,
 2.098343849182129,
 2.221309185028076,
 2.2919650077819824,
 2.1251535415649414,
 2.048246383666992,
 2.075944423675537,
 2.1724743843078613,
 2.122239589691162,
 2.3475935459136963,
 2.13238787651062,
 2.0909957885742188,
 2.1791422367095947,
 2.09989595413208,
 2.519381284713745,
 2.5619356632232666,
 2.889833450317383,
 2.8696420192718506,
 2.698503255844116,
 2.6929550170898438,
 2.7757112979888916,
 2.6663215160369873,
 2.638944625854492,
 2.9481515884399414,
 2.7254180908203125]

In [19]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 2.4065975109736124


In [20]:
total_training_loss_history

[{'loss': [0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.31422650814056396,
   0.314

In [21]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(30):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 6.5030828535556795
avg mae of training: 0.2233767718076706


In [22]:
total_scores

[[0.16850829124450684,
  0.16850829124450684,
  0.11325966566801071,
  0.41049760580062866,
  0.05978907272219658],
 [0.16850829124450684,
  0.16850829124450684,
  0.11325966566801071,
  0.41049760580062866,
  0.05978907272219658],
 [0.17077289521694183,
  0.17077289521694183,
  0.11670687794685364,
  0.4132467806339264,
  0.06111034378409386],
 [0.38291046023368835,
  0.38291046023368835,
  0.1649274080991745,
  0.6187976002693176,
  0.08945953845977783],
 [0.657734215259552,
  0.657734215259552,
  0.21416880190372467,
  0.8110081553459167,
  0.11442064493894577],
 [0.20070916414260864,
  0.20070916414260864,
  0.12875956296920776,
  0.4480057656764984,
  0.06900395452976227],
 [0.1733831912279129,
  0.1733831912279129,
  0.12914393842220306,
  0.41639307141304016,
  0.06335079669952393],
 [0.16850829124450684,
  0.16850829124450684,
  0.11325966566801071,
  0.41049760580062866,
  0.05978907272219658],
 [112.84567260742188,
  112.84567260742188,
  2.35500431060791,
  10.62288475036621

In [4]:
a = [[0.16850829124450684,
  0.16850829124450684,
  0.11325966566801071,
  0.41049760580062866,
  0.05978907272219658],
 [0.16850829124450684,
  0.16850829124450684,
  0.11325966566801071,
  0.41049760580062866,
  0.05978907272219658],
 [0.17077289521694183,
  0.17077289521694183,
  0.11670687794685364,
  0.4132467806339264,
  0.06111034378409386],
 [0.38291046023368835,
  0.38291046023368835,
  0.1649274080991745,
  0.6187976002693176,
  0.08945953845977783],
 [0.657734215259552,
  0.657734215259552,
  0.21416880190372467,
  0.8110081553459167,
  0.11442064493894577],
 [0.20070916414260864,
  0.20070916414260864,
  0.12875956296920776,
  0.4480057656764984,
  0.06900395452976227],
 [0.1733831912279129,
  0.1733831912279129,
  0.12914393842220306,
  0.41639307141304016,
  0.06335079669952393],
 [0.16850829124450684,
  0.16850829124450684,
  0.11325966566801071,
  0.41049760580062866,
  0.05978907272219658],
 [112.84567260742188,
  112.84567260742188,
  2.35500431060791,
  10.622884750366211,
  0.7484997510910034],
 [0.16850829124450684,
  0.16850829124450684,
  0.11325966566801071,
  0.41049760580062866,
  0.05978907272219658],
 [0.16850829124450684,
  0.16850829124450684,
  0.11325966566801071,
  0.41049760580062866,
  0.05978907272219658],
 [0.16850829124450684,
  0.16850829124450684,
  0.11325966566801071,
  0.41049760580062866,
  0.05978907272219658],
 [0.16850829124450684,
  0.16850829124450684,
  0.11325966566801071,
  0.41049760580062866,
  0.05978907272219658],
 [0.16850829124450684,
  0.16850829124450684,
  0.11325966566801071,
  0.41049760580062866,
  0.05978907272219658],
 [0.17553849518299103,
  0.17553849518299103,
  0.1258484572172165,
  0.4189731478691101,
  0.06400575488805771],
 [0.16850829124450684,
  0.16850829124450684,
  0.11325966566801071,
  0.41049760580062866,
  0.05978907272219658],
 [0.16850829124450684,
  0.16850829124450684,
  0.11325966566801071,
  0.41049760580062866,
  0.05978907272219658],
 [0.16850829124450684,
  0.16850829124450684,
  0.11325966566801071,
  0.41049760580062866,
  0.05978907272219658],
 [0.16850829124450684,
  0.16850829124450684,
  0.11325966566801071,
  0.41049760580062866,
  0.05978907272219658],
 [0.16850829124450684,
  0.16850829124450684,
  0.11325966566801071,
  0.41049760580062866,
  0.05978907272219658],
 [1.4428671598434448,
  1.4428671598434448,
  0.2734728455543518,
  1.2011940479278564,
  0.11370594799518585],
 [0.12899455428123474,
  0.12899455428123474,
  0.10588148236274719,
  0.35915812849998474,
  0.05133138969540596],
 [0.16850829124450684,
  0.16850829124450684,
  0.11325966566801071,
  0.41049760580062866,
  0.05978907272219658],
 [0.16853366792201996,
  0.16853366792201996,
  0.1135244071483612,
  0.4105285108089447,
  0.05981220677495003],
 [0.16850829124450684,
  0.16850829124450684,
  0.11325966566801071,
  0.41049760580062866,
  0.05978907272219658],
 [0.16850829124450684,
  0.16850829124450684,
  0.11325966566801071,
  0.41049760580062866,
  0.05978907272219658],
 [0.16850829124450684,
  0.16850829124450684,
  0.11325966566801071,
  0.41049760580062866,
  0.05978907272219658],
 [0.16850829124450684,
  0.16850829124450684,
  0.11325966566801071,
  0.41049760580062866,
  0.05978907272219658],
 [0.15608790516853333,
  0.15608790516853333,
  0.2223736196756363,
  0.3950796127319336,
  0.05756766349077225],
 [0.16918711364269257,
  0.16918711364269257,
  0.11479790508747101,
  0.411323606967926,
  0.06023918464779854]]

In [5]:
test_mae = []
for i in range(10):
    test_mae.append(a[i][2])
sum(test_mae)/10

0.35617495626211165

In [3]:
test_mae

[0.11325966566801071,
 0.11325966566801071,
 0.11670687794685364,
 0.1649274080991745,
 0.21416880190372467,
 0.12875956296920776,
 0.12914393842220306,
 0.11325966566801071,
 2.35500431060791,
 0.11325966566801071,
 0.11325966566801071,
 0.11325966566801071,
 0.11325966566801071,
 0.11325966566801071,
 0.1258484572172165,
 0.11325966566801071,
 0.11325966566801071,
 0.11325966566801071,
 0.11325966566801071,
 0.11325966566801071,
 0.2734728455543518,
 0.10588148236274719,
 0.11325966566801071,
 0.1135244071483612,
 0.11325966566801071,
 0.11325966566801071,
 0.11325966566801071,
 0.11325966566801071,
 0.2223736196756363,
 0.11479790508747101]

In [26]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(30):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 3.9901846890648205
avg mae: 0.20344278663396836
avg are: 0.12240638807415963


In [27]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 0.3288423418998718
median testing mse: 0.16850829124450684


In [28]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.1385248899459839
median testing mae: 0.11325966566801071


In [29]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.09116022288799286


In [30]:
test_ARE

[0.09116022,
 0.09116022,
 0.09116022,
 0.08839779,
 0.09944751,
 0.09116022,
 0.09116022,
 0.09116022,
 0.9857275,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.13029467,
 0.08931861,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022]

In [6]:
sum([0.09116022,
 0.09116022,
 0.09116022,
 0.08839779,
 0.09944751,
 0.09116022,
 0.09116022,
 0.09116022,
 0.9857275,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.13029467,
 0.08931861,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022,
 0.09116022][:10])/10

0.181169434